# Virtual Fields Method: Iosipescu Test for Orthotropic Materials
## Student Implementation Exercise

### Learning Objectives
- Understand the VFM principle for material identification
- Implement three different sets of virtual fields
- Analyze noise sensitivity through Monte Carlo simulation
- Compare different VF strategies

### Background
The Virtual Fields Method (VFM) allows identification of material parameters from full-field measurements.

**VFM Equation for orthotropic materials:**
$$Q_{11} \int_S \varepsilon_1 \varepsilon_1^* dS + Q_{22} \int_S \varepsilon_2 \varepsilon_2^* dS + Q_{12} \int_S (\varepsilon_1 \varepsilon_2^* + \varepsilon_2 \varepsilon_1^*) dS + Q_{66} \int_S \varepsilon_6 \varepsilon_6^* dS = \int_{L_f} T_i u_i^* dl$$

Where:
- $Q_{ij}$ are the stiffness parameters to identify
- $\varepsilon$ are the measured strain fields
- $\varepsilon^*$ are the virtual strain fields
- We need 4 independent virtual fields to solve for 4 unknowns

In [9]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import requests
from scipy import io
from io import StringIO

# Set plotting parameters
plt.rcParams['font.size'] = 12
%matplotlib inline

## Part 1: Class Provided Data Loading

In [10]:
class IosipescuVFMNoise:
    """
    VFM analysis for Iosipescu test with noise simulation
    """
    
    def __init__(self, data_source='csv'):
        self.data_source = data_source
        self.data = {}
        self.results = {}
        self.noise_results = {}
    
    def load_data_from_csv(self, csv_dir='.'):
        """Load data from CSV files (local or URL)"""

        # Determine if we're working with URL or local path
        is_url = csv_dir.startswith('http')

        # Construct paths
        if is_url:
            base = csv_dir.rstrip('/')
            scalar_path = f"{base}/scalarsFE.csv"
            fem_path = f"{base}/FEM2VFM.csv"
        else:
            scalar_path = os.path.join(csv_dir, 'scalarsFE.csv')
            fem_path = os.path.join(csv_dir, 'FEM2VFM.csv')

        # Load scalar parameters
        try:
            if is_url:
                response = requests.get(scalar_path)
                response.raise_for_status()
                content = response.text
            else:
                with open(scalar_path, 'r') as f:
                    content = f.read()

            for line in content.split('\n'):
                if '=' in line:
                    key, value = line.strip().split('=')
                    key = key.strip()
                    value = float(value.strip())

                    if key == 'Length':
                        self.data['L'] = value
                    elif key == 'Width':
                        self.data['w'] = value
                    elif key == 'Thick':
                        self.data['t'] = value
                    elif key == 'P':
                        self.data['F'] = value

            print(f"Loaded scalars from: {scalar_path}")
        except Exception as e:
            print(f"Error loading {scalar_path}: {e}")
            raise

        # Load FEM data
        try:
            fem_data = pd.read_csv(fem_path, sep=r'\s+')

            self.data['X1'] = fem_data['X_Coord'].values
            self.data['X2'] = fem_data['Y_Coord'].values
            self.data['Eps1'] = fem_data['Eps_X'].values
            self.data['Eps2'] = fem_data['Eps_Y'].values
            self.data['Eps6'] = fem_data['Eps_XY'].values

            print(f"Loaded FEM data from: {fem_path}")
        except Exception as e:
            print(f"Error loading {fem_path}: {e}")
            raise
    
    def _validate_data(self):
        """Validate loaded data (PROVIDED)"""
        print("Data validation:")
        for key, value in self.data.items():
            if np.isscalar(value):
                print(f"  {key}: {value}")
            else:
                arr = np.array(value)
                print(f"  {key}: shape {arr.shape}, range [{arr.min():.3f}, {arr.max():.3f}]")
        
        X1, X2 = self.data['X1'], self.data['X2']
        L, w = self.data['L'], self.data['w']
        print(f"\nCoordinate validation:")
        print(f"  X1 range: [{X1.min():.3f}, {X1.max():.3f}], Expected: [0, {L}]")
        print(f"  X2 range: [{X2.min():.3f}, {X2.max():.3f}], Expected: [approx ±{w/2}]")
    
    def load_data(self, source_path='.'):
        """Load data wrapper (PROVIDED)"""
        if self.data_source == 'csv':
            self.load_data_from_csv(source_path)
        else:
            raise ValueError("Only CSV loading implemented")

    def run_noise_analysis(self, noise_amplitude=10e-4, n_iterations=30, Q_ref=None, material_name="reference"):
        """
        Run VFM analysis with white noise simulation over multiple iterations

        Parameters:
        noise_amplitude: Standard deviation of Gaussian white noise (default: 10e-4)
        n_iterations: Number of Monte Carlo iterations (default: 30)
        Q_ref: Reference values for comparison (GPa)
        material_name: Name of the material for reference
        """
        print("=" * 80)
        print("VFM NOISE ANALYSIS - WHITE NOISE SIMULATION")
        print("=" * 80)
        print(f"Noise amplitude (std dev): {noise_amplitude:.1e}")
        print(f"Number of iterations: {n_iterations}")
        print(f"Material: {material_name}")

        # Get original strain data
        Eps1_orig = self.data['Eps1']
        Eps2_orig = self.data['Eps2']
        Eps6_orig = self.data['Eps6']

        print(f"\nOriginal strain data shapes:")
        print(f"  Eps1: {Eps1_orig.shape}")
        print(f"  Eps2: {Eps2_orig.shape}")
        print(f"  Eps6: {Eps6_orig.shape}")

        # Initialize result arrays
        Q11_set1, Q22_set1, Q12_set1, Q66_set1 = [], [], [], []
        Q11_set2, Q22_set2, Q12_set2, Q66_set2 = [], [], [], []
        Q11_set3, Q22_set3, Q12_set3, Q66_set3 = [], [], [], []

        print(f"\nRunning {n_iterations} iterations with white noise...")
        successful_iterations = 0

        for i in range(n_iterations):
            if (i + 1) % 10 == 0 or i == 0:
                print(f"  Iteration {i + 1}/{n_iterations}")

            # Add Gaussian white noise to strain data
            noise1 = np.random.randn(*Eps1_orig.shape) * noise_amplitude
            noise2 = np.random.randn(*Eps2_orig.shape) * noise_amplitude
            noise6 = np.random.randn(*Eps6_orig.shape) * noise_amplitude

            Eps1_noisy = Eps1_orig + noise1
            Eps2_noisy = Eps2_orig + noise2
            Eps6_noisy = Eps6_orig + noise6

            # Run all three virtual field sets
            try:
                # Set 1
                Q1 = self.virtual_fields_set_1(Eps1_noisy, Eps2_noisy, Eps6_noisy)
                Q11_set1.append(Q1[0])
                Q22_set1.append(Q1[1])
                Q12_set1.append(Q1[2])
                Q66_set1.append(Q1[3])

                # Set 2
                Q2 = self.virtual_fields_set_2(Eps1_noisy, Eps2_noisy, Eps6_noisy)
                Q11_set2.append(Q2[0])
                Q22_set2.append(Q2[1])
                Q12_set2.append(Q2[2])
                Q66_set2.append(Q2[3])

                # Set 3
                Q3 = self.virtual_fields_set_3(Eps1_noisy, Eps2_noisy, Eps6_noisy)
                Q11_set3.append(Q3[0])
                Q22_set3.append(Q3[1])
                Q12_set3.append(Q3[2])
                Q66_set3.append(Q3[3])

                successful_iterations += 1

            except np.linalg.LinAlgError:
                print(f"    Warning: Singular matrix at iteration {i + 1}, skipping...")
                continue

        print(f"Completed {successful_iterations}/{n_iterations} successful iterations")

        # Convert to numpy arrays and to GPa
        Q11_set1 = np.array(Q11_set1) / 1e3
        Q22_set1 = np.array(Q22_set1) / 1e3
        Q12_set1 = np.array(Q12_set1) / 1e3
        Q66_set1 = np.array(Q66_set1) / 1e3

        Q11_set2 = np.array(Q11_set2) / 1e3
        Q22_set2 = np.array(Q22_set2) / 1e3
        Q12_set2 = np.array(Q12_set2) / 1e3
        Q66_set2 = np.array(Q66_set2) / 1e3

        Q11_set3 = np.array(Q11_set3) / 1e3
        Q22_set3 = np.array(Q22_set3) / 1e3
        Q12_set3 = np.array(Q12_set3) / 1e3
        Q66_set3 = np.array(Q66_set3) / 1e3

        # Store results
        self.noise_results = {
            'set1': {'Q11': Q11_set1, 'Q22': Q22_set1, 'Q12': Q12_set1, 'Q66': Q66_set1},
            'set2': {'Q11': Q11_set2, 'Q22': Q22_set2, 'Q12': Q12_set2, 'Q66': Q66_set2},
            'set3': {'Q11': Q11_set3, 'Q22': Q22_set3, 'Q12': Q12_set3, 'Q66': Q66_set3}
        }

        # Store the last iteration's noisy strain data for plotting
        self.Eps1_noisy = Eps1_noisy
        self.Eps2_noisy = Eps2_noisy
        self.Eps6_noisy = Eps6_noisy

        # Calculate and display statistics
        self._display_statistics(Q_ref, material_name)

        return self.noise_results

    def _display_statistics(self, Q_ref=None, material_name="reference"):
        """Display statistical results"""
        print("\n" + "=" * 90)
        print("STATISTICAL RESULTS")
        print("=" * 90)

        # Parameter names
        params = ['Q11', 'Q22', 'Q12', 'Q66']

        # Display results for each virtual field set
        for set_num, set_name in enumerate(['set1', 'set2', 'set3'], 1):
            print(f"\nVirtual Field Set {set_num}:")
            print("-" * 70)
            print(f"{'Param':<6} {'Mean':<10} {'Std':<10} {'CV%':<10}", end="")
            if Q_ref is not None:
                print(f" {'Ref':<10} {'Error%':<10}")
            else:
                print()
            print("-" * 70)

            for i, param in enumerate(params):
                values = self.noise_results[set_name][param]
                mean_val = np.mean(values)
                std_val = np.std(values)
                cv_val = (std_val / mean_val) * 100 if mean_val != 0 else 0

                print(f"{param:<6} {mean_val:<10.3f} {std_val:<10.4f} {cv_val:<10.3f}", end="")

                if Q_ref is not None:
                    error_pct = abs(mean_val - Q_ref[i]) / Q_ref[i] * 100 if Q_ref[i] != 0 else 0
                    print(f" {Q_ref[i]:<10.3f} {error_pct:<10.3f}")
                else:
                    print()

        # Display noise robustness comparison
        print(f"\n" + "=" * 70)
        print("NOISE ROBUSTNESS COMPARISON (Coefficient of Variation %)")
        print("=" * 70)
        print(f"{'Param':<8} {'Set 1':<12} {'Set 2':<12} {'Set 3':<12} {'Best Set':<10}")
        print("-" * 70)

        for i, param in enumerate(params):
            cv1 = (np.std(self.noise_results['set1'][param]) / np.mean(self.noise_results['set1'][param])) * 100
            cv2 = (np.std(self.noise_results['set2'][param]) / np.mean(self.noise_results['set2'][param])) * 100
            cv3 = (np.std(self.noise_results['set3'][param]) / np.mean(self.noise_results['set3'][param])) * 100

            cvs = [cv1, cv2, cv3]
            best_set = np.argmin(cvs) + 1

            print(f"{param:<8} {cv1:<12.3f} {cv2:<12.3f} {cv3:<12.3f} Set {best_set:<6}")

        # Overall summary
        print(f"\n" + "=" * 70)
        print("SUMMARY COMPARISON TABLE (GPa)")
        print("=" * 70)
        print(f"{'Parameter':<10} {'Set 1':<12} {'Set 2':<12} {'Set 3':<12}", end="")
        if Q_ref is not None:
            print(f" {'Reference':<12}")
        else:
            print()
        print("-" * 70)

        for i, param in enumerate(params):
            mean1 = np.mean(self.noise_results['set1'][param])
            mean2 = np.mean(self.noise_results['set2'][param])
            mean3 = np.mean(self.noise_results['set3'][param])

            print(f"{param:<10} {mean1:<12.3f} {mean2:<12.3f} {mean3:<12.3f}", end="")
            if Q_ref is not None:
                print(f" {Q_ref[i]:<12.3f}")
            else:
                print()

    def plot_results(self, save_plots=True):
        """Plot histograms of identified parameters"""
        if not self.noise_results:
            print("No noise analysis results to plot. Run noise analysis first.")
            return

        params = ['Q11', 'Q22', 'Q12', 'Q66']
        param_units = ['GPa', 'GPa', 'GPa', 'GPa']

        # Set up plotting parameters
        plt.rcParams['font.size'] = 12
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        axes = axes.ravel()

        colors = ['blue', 'red', 'green']
        alphas = [0.6, 0.6, 0.6]

        for i, param in enumerate(params):
            ax = axes[i]

            # Plot histograms for all three sets
            ax.hist(self.noise_results['set1'][param], alpha=alphas[0], label='Set 1',
                    bins=20, density=True, color=colors[0], edgecolor='black', linewidth=0.5)
            ax.hist(self.noise_results['set2'][param], alpha=alphas[1], label='Set 2',
                    bins=20, density=True, color=colors[1], edgecolor='black', linewidth=0.5)
            ax.hist(self.noise_results['set3'][param], alpha=alphas[2], label='Set 3',
                    bins=20, density=True, color=colors[2], edgecolor='black', linewidth=0.5)

            ax.set_xlabel(f'{param} ({param_units[i]})', fontsize=14)
            ax.set_ylabel('Probability Density', fontsize=14)
            ax.set_title(f'Distribution of {param} with White Noise', fontsize=16, fontweight='bold')
            ax.legend(fontsize=12)
            ax.grid(True, alpha=0.3)

            # Add statistics text box
            mean1 = np.mean(self.noise_results['set1'][param])
            mean2 = np.mean(self.noise_results['set2'][param])
            mean3 = np.mean(self.noise_results['set3'][param])

            std1 = np.std(self.noise_results['set1'][param])
            std2 = np.std(self.noise_results['set2'][param])
            std3 = np.std(self.noise_results['set3'][param])

            stats_text = f'Set 1: μ={mean1:.3f}, σ={std1:.4f}\n'
            stats_text += f'Set 2: μ={mean2:.3f}, σ={std2:.4f}\n'
            stats_text += f'Set 3: μ={mean3:.3f}, σ={std3:.4f}'

            ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=10,
                    verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

        plt.tight_layout()

        if save_plots:
            plt.savefig('vfm_noise_analysis_results.png', dpi=300, bbox_inches='tight',
                        facecolor='white', edgecolor='none')
            print("Histogram plot saved as 'vfm_noise_analysis_results.png'")

        plt.show()

    def plot_strain_fields(self, save_path='strain_fields_with_noise.png'):
        """Plot the strain field distributions with noise"""
        if not hasattr(self, 'Eps1_noisy'):
            print("No noisy strain data available. Run noise analysis first.")
            return

        FS = 22
        cor = 'BrBG'

        # Disable LaTeX rendering in Jupyter
        plt.rcParams['text.usetex'] = False
        plt.rcParams['font.size'] = FS

        # Get data
        X1 = self.data['X1'].flatten()
        X2 = self.data['X2'].flatten()
        strain_data = [self.Eps1_noisy.flatten(), self.Eps2_noisy.flatten(), self.Eps6_noisy.flatten()]

        # Use simple labels without LaTeX (or use matplotlib's mathtext)
        strain_labels = ['ε₁ (with noise)', 'ε₂ (with noise)', 'ε₆ (with noise)']
        # Or use matplotlib's mathtext (doesn't require LaTeX):
        # strain_labels = [r'$\varepsilon_1$ (with noise)', r'$\varepsilon_2$ (with noise)', r'$\varepsilon_6$ (with noise)']

        # Create figure
        fig, axes = plt.subplots(1, 3, figsize=(20, 6))

        # Plot each strain component
        for i, (data, label, ax) in enumerate(zip(strain_data, strain_labels, axes)):
            vmin, vmax = np.min(data), np.max(data)
            im = ax.tricontourf(X1, X2, data, levels=20, cmap=cor, vmin=vmin, vmax=vmax)
            ax.set_aspect('equal')
            ax.set_title(label, fontsize=FS + 2)
            ax.set_xlabel('x₁ (mm)', fontsize=FS)  # or r'$x_1$ (mm)' with mathtext
            ax.set_ylabel('x₂ (mm)', fontsize=FS)  # or r'$x_2$ (mm)' with mathtext
            ax.tick_params(labelsize=FS - 4)

            cbar = plt.colorbar(im, ax=ax, shrink=0.6, aspect=30, pad=0.05, fraction=0.04)
            cbar.ax.tick_params(labelsize=FS - 4)
            cbar.formatter.set_powerlimits((0, 0))
            cbar.update_ticks()

        plt.tight_layout()
        plt.savefig(save_path, dpi=150, bbox_inches='tight',
                    facecolor='white', edgecolor='none')
        print(f"Strain field plot saved as '{save_path}'")
        plt.show()

    def run_clean_analysis_comparison(self, Q_ref=None, material_name="reference"):
        """Run analysis without noise for comparison"""
        print("\n" + "=" * 80)
        print("CLEAN DATA ANALYSIS (NO NOISE) FOR COMPARISON")
        print("=" * 80)

        # Get original strain data
        Eps1_orig = self.data['Eps1']
        Eps2_orig = self.data['Eps2']
        Eps6_orig = self.data['Eps6']

        try:
            # Run all three virtual field sets with clean data
            Q1_clean = self.virtual_fields_set_1(Eps1_orig, Eps2_orig, Eps6_orig)
            Q2_clean = self.virtual_fields_set_2(Eps1_orig, Eps2_orig, Eps6_orig)
            Q3_clean = self.virtual_fields_set_3(Eps1_orig, Eps2_orig, Eps6_orig)

            # Convert to GPa
            Q1_clean_gpa = Q1_clean / 1e3
            Q2_clean_gpa = Q2_clean / 1e3
            Q3_clean_gpa = Q3_clean / 1e3

            print("Clean data results (GPa):")
            print("-" * 70)
            print(f"{'Param':<8} {'Set 1':<12} {'Set 2':<12} {'Set 3':<12}", end="")
            if Q_ref is not None:
                print(f" {'Reference':<12}")
            else:
                print()
            print("-" * 70)

            params = ['Q11', 'Q22', 'Q12', 'Q66']
            for i, param in enumerate(params):
                print(f"{param:<8} {Q1_clean_gpa[i]:<12.3f} {Q2_clean_gpa[i]:<12.3f} {Q3_clean_gpa[i]:<12.3f}", end="")
                if Q_ref is not None:
                    print(f" {Q_ref[i]:<12.3f}")
                else:
                    print()

            return {'set1': Q1_clean_gpa, 'set2': Q2_clean_gpa, 'set3': Q3_clean_gpa}

        except Exception as e:
            print(f"Error in clean analysis: {e}")
            return None

## 3. Configuration Parameters

In [11]:
# Noise analysis parameters
noise_amplitude = 10e-4  # Standard deviation of Gaussian white noise
n_iterations = 30        # Number of Monte Carlo iterations

# Reference values (in GPa)
Q_ref = np.array([15.536, 1.965, 0.926, 1.109])  # Q11, Q22, Q12, Q66
material_name = "Wood (from working VFM code)"

# Display configuration
print("=" * 60)
print("NOISE ANALYSIS CONFIGURATION")
print("=" * 60)
print(f"Noise amplitude (std dev): {noise_amplitude:.1e}")
print(f"Monte Carlo iterations: {n_iterations}")
print(f"Material: {material_name}")
print(f"Reference values (GPa): Q11={Q_ref[0]:.3f}, Q22={Q_ref[1]:.3f}, Q12={Q_ref[2]:.3f}, Q66={Q_ref[3]:.3f}")

NOISE ANALYSIS CONFIGURATION
Noise amplitude (std dev): 1.0e-03
Monte Carlo iterations: 30
Material: Wood (from working VFM code)
Reference values (GPa): Q11=15.536, Q22=1.965, Q12=0.926, Q66=1.109


## 4. Load Data

In [12]:
# Initialize noise analysis
vfm_noise = IosipescuVFMNoise(data_source='csv')

# Base URL
BASE_URL = 'https://userweb.fct.unl.pt/~jmc.xavier/CISM_C2516/1_VFM_Iosipescu_orthotropic_manuallyVFs/'

# Load data from current directory
# vfm_noise.load_data_from_csv('.')

try:
    vfm_noise.load_data_from_csv(BASE_URL)
    print("Data loaded successfully from URL!")
except Exception as e:
    print(f"Error: {e}")

# Still works with local files
# vfm_noise.load_data_from_csv('.')
# try:
#     vfm_noise.load_data('.')  # Load from current directory
#     print("\nData loaded successfully!")
# except Exception as e:
#     print(f"Error loading data: {e}")
#     print("Make sure 'scalarsFE.csv' and 'FEM2VFM.csv' are in the current directory")
#


Loaded scalars from: https://userweb.fct.unl.pt/~jmc.xavier/CISM_C2516/1_VFM_Iosipescu_orthotropic_manuallyVFs/scalarsFE.csv
Loaded FEM data from: https://userweb.fct.unl.pt/~jmc.xavier/CISM_C2516/1_VFM_Iosipescu_orthotropic_manuallyVFs/FEM2VFM.csv
Data loaded successfully from URL!


## Part 2: Implement Virtual Field Sets

### Virtual Field Set 1

You need to implement the first set of virtual fields:

**Displacement fields:**
- VF1: $u_1^* = 0, \quad u_2^* = -x_1$
- VF2: $u_1^* = x_1(L-x_1)x_2, \quad u_2^* = \frac{x_1^3}{3} - \frac{Lx_1^2}{2}$
- VF3: $u_1^* = 0, \quad u_2^* = x_1(L-x_1)x_2$
- VF4: $u_1^* = \frac{L}{2\pi}\sin(2\pi x_1/L), \quad u_2^* = 0$

**Corresponding strain fields:**
- VF1: $\varepsilon_1^* = 0, \quad \varepsilon_2^* = 0, \quad \varepsilon_6^* = -1$
- VF2: $\varepsilon_1^* = (L-2x_1)x_2, \quad \varepsilon_2^* = 0, \quad \varepsilon_6^* = 0$
- VF3: $\varepsilon_1^* = 0, \quad \varepsilon_2^* = x_1(L-x_1), \quad \varepsilon_6^* = (L-2x_1)x_2$
- VF4: $\varepsilon_1^* = \cos(2\pi x_1/L), \quad \varepsilon_2^* = 0, \quad \varepsilon_6^* = 0$

In [ ]:
def virtual_fields_set_1(self, Eps1, Eps2, Eps6):
    """
    First set of virtual fields
    
    TODO: Implement this method
    
    Steps:
    1. Extract coordinates and geometry from self.data
    2. Initialize 4x4 matrix A and 4x1 vector B
    3. For each virtual field, compute the contributions to A and B
    4. Solve the system AQ = B to get [Q11, Q22, Q12, Q66]
    
    Hints:
    - Use np.mean() to compute spatial averages (integral approximation)
    - Right-hand side B comes from external work: F/(w*t) or F*L²/(6*w*t)
    - Use np.linalg.solve(A, B) to solve the system
    
    Returns:
    -------
    Q : ndarray
        Array [Q11, Q22, Q12, Q66] in MPa
    """
    # Get data
    X1 = self.data['X1']
    X2 = self.data['X2']
    L = self.data['L']
    w = self.data['w']
    t = self.data['t']
    F = self.data['F']
    
    # Initialize system
    A = np.zeros((4, 4))
    B = np.zeros(4)
    
    # ==========================================
    # TODO: Implement VF1
    # ε1* = 0, ε2* = 0, ε6* = -1
    # ==========================================
    # A[0, 0] = ?  # Q11 coefficient
    # A[0, 1] = ?  # Q22 coefficient
    # A[0, 2] = ?  # Q12 coefficient
    # A[0, 3] = ?  # Q66 coefficient
    # B[0] = ?
    
    # ==========================================
    # TODO: Implement VF2
    # ε1* = (L-2x1)x2, ε2* = 0, ε6* = 0
    # ==========================================
    # vf2_eps1_star = ?
    # A[1, 0] = ?
    # A[1, 1] = ?
    # A[1, 2] = ?
    # A[1, 3] = ?
    # B[1] = ?
    
    # ==========================================
    # TODO: Implement VF3
    # ε1* = 0, ε2* = x1(L-x1), ε6* = (L-2x1)x2
    # ==========================================
    
    # ==========================================
    # TODO: Implement VF4
    # ε1* = cos(2πx1/L), ε2* = 0, ε6* = 0
    # ==========================================
    
    # Solve system
    # TODO: Q = np.linalg.solve(A, B)
    
    raise NotImplementedError("You need to implement this method!")
    # return Q

# Add method to class
IosipescuVFM.virtual_fields_set_1 = virtual_fields_set_1

### Virtual Field Set 2 (Modified 4th field)

**TODO:** Implement Set 2 with modified VF4:
- VF4: $u_1^* = 0, \quad u_2^* = x_1(L-x_1)x_2^3$
- VF4 strains: $\varepsilon_1^* = 0, \quad \varepsilon_2^* = 3x_1(L-x_1)x_2^2, \quad \varepsilon_6^* = (L-2x_1)x_2^3$

In [ ]:
def virtual_fields_set_2(self, Eps1, Eps2, Eps6):
    """
    Second set of virtual fields (modified 4th field)
    
    TODO: Implement this method
    Hint: VF1-3 are the same as Set 1, only VF4 changes
    """
    raise NotImplementedError("You need to implement this method!")

IosipescuVFM.virtual_fields_set_2 = virtual_fields_set_2

### Virtual Field Set 3 (Modified 1st field)

**TODO:** Implement Set 3 with modified VF1:
- VF1: $u_1^* = 0, \quad u_2^* = -x_1^3$
- VF1 strains: $\varepsilon_1^* = 0, \quad \varepsilon_2^* = -3x_1^2, \quad \varepsilon_6^* = 0$

In [ ]:
def virtual_fields_set_3(self, Eps1, Eps2, Eps6):
    """
    Third set of virtual fields (modified 1st field)
    
    TODO: Implement this method
    """
    raise NotImplementedError("You need to implement this method!")

IosipescuVFM.virtual_fields_set_3 = virtual_fields_set_3

## Part 3: Test Your Implementation (Clean Data)

Once you've implemented the virtual field methods, test them with clean data:

In [ ]:
def run_clean_analysis(self, Q_ref=None, material_name="reference"):
    """Run analysis without noise (PROVIDED)"""
    print("\n" + "="*80)
    print("CLEAN DATA ANALYSIS")
    print("="*80)
    
    Eps1 = self.data['Eps1']
    Eps2 = self.data['Eps2']
    Eps6 = self.data['Eps6']
    
    try:
        Q1 = self.virtual_fields_set_1(Eps1, Eps2, Eps6)
        Q2 = self.virtual_fields_set_2(Eps1, Eps2, Eps6)
        Q3 = self.virtual_fields_set_3(Eps1, Eps2, Eps6)
        
        # Convert to GPa
        Q1_gpa = Q1 / 1e3
        Q2_gpa = Q2 / 1e3
        Q3_gpa = Q3 / 1e3
        
        print("\nResults (GPa):")
        print("-"*70)
        print(f"{'Param':<8} {'Set 1':<12} {'Set 2':<12} {'Set 3':<12}", end="")
        if Q_ref is not None:
            print(f" {'Reference':<12}")
        else:
            print()
        print("-"*70)
        
        params = ['Q11', 'Q22', 'Q12', 'Q66']
        for i, param in enumerate(params):
            print(f"{param:<8} {Q1_gpa[i]:<12.3f} {Q2_gpa[i]:<12.3f} {Q3_gpa[i]:<12.3f}", end="")
            if Q_ref is not None:
                print(f" {Q_ref[i]:<12.3f}")
            else:
                print()
        
        return {'set1': Q1_gpa, 'set2': Q2_gpa, 'set3': Q3_gpa}
        
    except NotImplementedError as e:
        print(f"\nError: {e}")
        print("Please implement the virtual field methods first!")
        return None

IosipescuVFM.run_clean_analysis = run_clean_analysis

In [ ]:
# Reference values from FE model (GPa)
Q_ref = np.array([15.536, 1.965, 0.926, 1.109])

# Test your implementation
clean_results = vfm.run_clean_analysis(Q_ref=Q_ref, material_name="Wood")

### Expected Results:

Your implementation should give results close to:
- Q11 ≈ 15.5 GPa
- Q22 ≈ 1.96 GPa  
- Q12 ≈ 0.92-0.98 GPa (varies by set)
- Q66 ≈ 1.11 GPa

If your results are significantly different, check:
1. Sign conventions in the virtual strains
2. Averaging using `np.mean()`
3. Right-hand side values (external work)

## Part 4: Noise Analysis

This section analyzes robustness to measurement noise using Monte Carlo simulation.

In [ ]:
def run_noise_analysis(self, noise_amplitude=10e-4, n_iterations=30, Q_ref=None):
    """Monte Carlo noise analysis (PROVIDED)"""
    print("\n" + "="*80)
    print("NOISE SENSITIVITY ANALYSIS")
    print("="*80)
    print(f"Noise amplitude (std dev): {noise_amplitude:.1e}")
    print(f"Number of iterations: {n_iterations}")
    
    Eps1_orig = self.data['Eps1']
    Eps2_orig = self.data['Eps2']
    Eps6_orig = self.data['Eps6']
    
    # Initialize result arrays
    Q11_set1, Q22_set1, Q12_set1, Q66_set1 = [], [], [], []
    Q11_set2, Q22_set2, Q12_set2, Q66_set2 = [], [], [], []
    Q11_set3, Q22_set3, Q12_set3, Q66_set3 = [], [], [], []
    
    print(f"\nRunning {n_iterations} iterations...")
    
    for i in range(n_iterations):
        if (i + 1) % 10 == 0:
            print(f"  Iteration {i+1}/{n_iterations}")
        
        # Add Gaussian white noise
        noise1 = np.random.randn(*Eps1_orig.shape) * noise_amplitude
        noise2 = np.random.randn(*Eps2_orig.shape) * noise_amplitude
        noise6 = np.random.randn(*Eps6_orig.shape) * noise_amplitude
        
        Eps1_noisy = Eps1_orig + noise1
        Eps2_noisy = Eps2_orig + noise2
        Eps6_noisy = Eps6_orig + noise6
        
        try:
            Q1 = self.virtual_fields_set_1(Eps1_noisy, Eps2_noisy, Eps6_noisy)
            Q11_set1.append(Q1[0]); Q22_set1.append(Q1[1])
            Q12_set1.append(Q1[2]); Q66_set1.append(Q1[3])
            
            Q2 = self.virtual_fields_set_2(Eps1_noisy, Eps2_noisy, Eps6_noisy)
            Q11_set2.append(Q2[0]); Q22_set2.append(Q2[1])
            Q12_set2.append(Q2[2]); Q66_set2.append(Q2[3])
            
            Q3 = self.virtual_fields_set_3(Eps1_noisy, Eps2_noisy, Eps6_noisy)
            Q11_set3.append(Q3[0]); Q22_set3.append(Q3[1])
            Q12_set3.append(Q3[2]); Q66_set3.append(Q3[3])
            
        except (np.linalg.LinAlgError, NotImplementedError):
            continue
    
    # Convert to GPa and store
    self.noise_results = {
        'set1': {
            'Q11': np.array(Q11_set1)/1e3, 'Q22': np.array(Q22_set1)/1e3,
            'Q12': np.array(Q12_set1)/1e3, 'Q66': np.array(Q66_set1)/1e3
        },
        'set2': {
            'Q11': np.array(Q11_set2)/1e3, 'Q22': np.array(Q22_set2)/1e3,
            'Q12': np.array(Q12_set2)/1e3, 'Q66': np.array(Q66_set2)/1e3
        },
        'set3': {
            'Q11': np.array(Q11_set3)/1e3, 'Q22': np.array(Q22_set3)/1e3,
            'Q12': np.array(Q12_set3)/1e3, 'Q66': np.array(Q66_set3)/1e3
        }
    }
    
    self._display_noise_statistics(Q_ref)
    return self.noise_results

def _display_noise_statistics(self, Q_ref=None):
    """Display noise analysis statistics (PROVIDED)"""
    print("\n" + "="*90)
    print("STATISTICAL RESULTS")
    print("="*90)
    
    params = ['Q11', 'Q22', 'Q12', 'Q66']
    
    for set_num, set_name in enumerate(['set1', 'set2', 'set3'], 1):
        print(f"\nVirtual Field Set {set_num}:")
        print("-"*70)
        print(f"{'Param':<6} {'Mean':<10} {'Std':<10} {'CV%':<10}", end="")
        if Q_ref is not None:
            print(f" {'Ref':<10} {'Error%':<10}")
        else:
            print()
        print("-"*70)
        
        for i, param in enumerate(params):
            values = self.noise_results[set_name][param]
            mean_val = np.mean(values)
            std_val = np.std(values)
            cv_val = (std_val / mean_val) * 100 if mean_val != 0 else 0
            
            print(f"{param:<6} {mean_val:<10.3f} {std_val:<10.4f} {cv_val:<10.3f}", end="")
            
            if Q_ref is not None:
                error_pct = abs(mean_val - Q_ref[i]) / Q_ref[i] * 100
                print(f" {Q_ref[i]:<10.3f} {error_pct:<10.3f}")
            else:
                print()

IosipescuVFM.run_noise_analysis = run_noise_analysis
IosipescuVFM._display_noise_statistics = _display_noise_statistics

In [ ]:
# Run noise analysis
noise_results = vfm.run_noise_analysis(
    noise_amplitude=10e-4,
    n_iterations=30,
    Q_ref=Q_ref
)

## Part 5: Visualization

In [ ]:
def plot_noise_results(self):
    """Plot histogram of identified parameters (PROVIDED)"""
    if not self.noise_results:
        print("No results to plot. Run noise analysis first.")
        return
    
    params = ['Q11', 'Q22', 'Q12', 'Q66']
    param_units = ['GPa', 'GPa', 'GPa', 'GPa']
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()
    
    colors = ['blue', 'red', 'green']
    
    for i, param in enumerate(params):
        ax = axes[i]
        
        for j, set_name in enumerate(['set1', 'set2', 'set3']):
            data = self.noise_results[set_name][param]
            ax.hist(data, alpha=0.6, label=f'Set {j+1}',
                   bins=20, density=True, color=colors[j],
                   edgecolor='black', linewidth=0.5)
        
        ax.set_xlabel(f'{param} ({param_units[i]})', fontsize=14)
        ax.set_ylabel('Probability Density', fontsize=14)
        ax.set_title(f'Distribution of {param}', fontsize=16, fontweight='bold')
        ax.legend(fontsize=12)
        ax.grid(True, alpha=0.3)
        
        # Add statistics text
        stats_text = ''
        for j, set_name in enumerate(['set1', 'set2', 'set3']):
            data = self.noise_results[set_name][param]
            stats_text += f'Set {j+1}: μ={np.mean(data):.3f}, σ={np.std(data):.4f}\n'
        
        ax.text(0.02, 0.98, stats_text, transform=ax.transAxes,
               fontsize=10, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

IosipescuVFM.plot_noise_results = plot_noise_results

In [ ]:
# Plot results
vfm.plot_noise_results()

## Discussion Questions

1. **Which virtual field set gives the most accurate Q12 identification? Why?**

2. **Which parameter shows the highest noise sensitivity (highest CV%)? Can you explain why based on the strain field distribution?**

3. **Compare the three sets: What are the trade-offs between them?**

4. **How would you design a better virtual field if you could choose any kinematically admissible field?**

## Bonus Challenges

If you finish early, try:

1. **Implement a 4th virtual field set** with your own choice of fields
2. **Study the effect of noise amplitude** - how does CV% scale with noise?
3. **Compute the condition number** of matrix A for each set - does it correlate with noise sensitivity?